### Predict marathon time given (incomplete) 5k, 10k and half times

In [153]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [154]:
# Load data
file='data.csv'
data = pd.read_csv(file, index_col=0)
data.head()

,adjusted,age,bmi,cohort1,cohort2,cohort3,cohort4,endurancecat,endurancespeed,female,...,mf_di,mf_ti,mf_ti_adj,mf_tr,max,sprint,tempo,typical,model1_time,model2_time
id,,,,,,,,,,,,,,,,,,,,,
1,1,25.0,21.622116,NaN,NaN,NaN,NaN,1,2,1,...,NaN,NaN,NaN,NaN,47.0,1,0,30.0,NaN,NaN
2,1,41.0,23.905970,NaN,NaN,NaN,NaN,1,2,0,...,NaN,NaN,NaN,NaN,28.0,1,1,23.0,NaN,NaN
3,1,25.0,21.640728,NaN,NaN,NaN,NaN,2,3,0,...,NaN,NaN,NaN,NaN,70.0,1,1,50.0,NaN,NaN
4,1,35.0,23.592323,1.0,NaN,NaN,NaN,2,5,0,...,4.0,10295.0,10399.02832,1.0,60.0,1,1,50.0,NaN,NaN
5,1,34.0,22.706404,NaN,NaN,NaN,NaN,2,3,0,...,NaN,NaN,NaN,NaN,15.0,0,0,8.0,NaN,NaN


In [155]:
# Filter by:
# adjusted=1 -- not enough races otherwise
# cohort1 == 1 -- otherwise no marathon time
data = data[data['adjusted']==1]
data = data[data['cohort1']==1]
data.head()

,adjusted,age,bmi,cohort1,cohort2,cohort3,cohort4,endurancecat,endurancespeed,female,...,mf_di,mf_ti,mf_ti_adj,mf_tr,max,sprint,tempo,typical,model1_time,model2_time
id,,,,,,,,,,,,,,,,,,,,,
4,1,35.0,23.592323,1.0,NaN,NaN,NaN,2,5,0,...,4.0,10295.0,10399.028320,1.0,60.0,1,1,50.0,NaN,NaN
7,1,33.0,22.518295,1.0,1.0,1.0,1.0,2,3,0,...,3.0,12292.0,12292.000000,2.0,50.0,0,0,40.0,NaN,NaN
12,1,38.0,25.560312,1.0,NaN,NaN,NaN,1,2,0,...,4.0,10980.0,11098.411133,1.0,65.0,0,1,50.0,206.163071,NaN
13,1,34.0,22.607931,1.0,1.0,NaN,NaN,1,2,0,...,3.0,10694.0,10694.000000,2.0,88.0,1,1,80.0,NaN,NaN
14,1,39.0,24.974836,1.0,1.0,1.0,1.0,3,6,0,...,2.0,13452.0,13387.699219,1.0,51.0,0,1,40.0,NaN,NaN


In [156]:
# Drop unwanted columns: 
# endurancespeed, 5 mile and 10 mile data (not enough), predicted results from other models
# Calculated data for 5k, 10k, half and full
data.drop(['endurancespeed', 'm5_d', 'm5_di', 'm5_ti', 'm5_ti_adj', 'm5_tr', \
           'm10_d', 'm10_di', 'm10_ti', 'm10_ti_adj', 'm10_tr', \
           'k5_ti_adj', 'k10_ti_adj', 'mh_ti_adj', 'mf_ti_adj', \
           'adjusted', 'cohort1', 'cohort2', 'cohort3', 'cohort4', \
          'model1_time', 'model2_time', 'group'], axis=1, inplace=True)
data.head()

,age,bmi,endurancecat,female,footwear,injury,k5_d,k5_di,k5_ti,k5_tr,...,mh_ti,mh_tr,mf_d,mf_di,mf_ti,mf_tr,max,sprint,tempo,typical
id,,,,,,,,,,,,,,,,,,,,,
4,35.0,23.592323,2,0,2,2,5000.0,3.0,1075.0,1.0,...,NaN,NaN,42195.0,4.0,10295.0,1.0,60.0,1,1,50.0
7,33.0,22.518295,2,0,2,2,5000.0,4.0,1292.0,3.0,...,5760.0,2.0,42195.0,3.0,12292.0,2.0,50.0,0,0,40.0
12,38.0,25.560312,1,0,2,1,NaN,NaN,NaN,NaN,...,NaN,NaN,42195.0,4.0,10980.0,1.0,65.0,0,1,50.0
13,34.0,22.607931,1,0,2,1,NaN,NaN,NaN,NaN,...,4995.0,2.0,42195.0,3.0,10694.0,2.0,88.0,1,1,80.0
14,39.0,24.974836,3,0,2,1,5000.0,4.0,1222.0,1.0,...,5884.0,1.0,42195.0,2.0,13452.0,1.0,51.0,0,1,40.0


In [157]:
data.dtypes

age             float64
bmi             float64
endurancecat      int64
female            int64
footwear          int64
injury            int64
k5_d            float64
k5_di           float64
k5_ti           float64
k5_tr           float64
k10_d           float64
k10_di          float64
k10_ti          float64
k10_tr          float64
mh_d            float64
mh_di           float64
mh_ti           float64
mh_tr           float64
mf_d            float64
mf_di           float64
mf_ti           float64
mf_tr           float64
max             float64
sprint            int64
tempo             int64
typical         float64
dtype: object

In [158]:
# Compute additional features
# Compute speed (then drop time and distance)
data['k5_s'] = data['k5_d']/data['k5_ti']
data['k10_s'] = data['k10_d']/data['k10_ti']
data['mh_s'] = data['mh_d']/data['mh_ti']
data['mf_s'] = data['mf_d']/data['mf_ti']

In [159]:
# Drop any more fields
data.drop(['k5_d', 'k5_ti', 'k10_d', 'k10_ti', 'mh_d', 'mh_ti', 'mf_d', 'mf_ti'], axis=1, inplace=True)

In [160]:
data.describe()

,age,bmi,endurancecat,female,footwear,injury,k5_di,k5_tr,k10_di,k10_tr,...,mf_di,mf_tr,max,sprint,tempo,typical,k5_s,k10_s,mh_s,mf_s
count,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,397.000000,397.000000,268.000000,268.000000,...,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,397.000000,268.000000,654.000000,929.000000
mean,37.328310,23.378063,1.752422,0.361679,1.799785,1.444564,3.193955,1.896725,3.089552,1.940299,...,2.939720,1.684607,53.001292,0.566200,0.669537,41.648547,4.038554,3.763046,3.565577,3.254957
std,9.093911,2.854111,0.655758,0.480745,0.433957,0.657676,0.843971,0.726076,0.820730,0.762032,...,0.764673,0.647101,19.135593,0.495865,0.470633,16.049241,0.638589,0.648497,0.602743,0.596153
min,20.000000,17.530680,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,1.000000,3.000000,0.000000,0.000000,10.000000,1.941748,1.773050,2.129770,1.826623
25%,31.000000,21.521336,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,3.000000,1.000000,...,2.000000,1.000000,40.000000,0.000000,0.000000,30.000000,3.673769,3.340018,3.160319,2.847166
50%,35.000000,23.083443,2.000000,0.000000,2.000000,1.000000,3.000000,2.000000,3.000000,2.000000,...,3.000000,2.000000,50.000000,1.000000,1.000000,40.000000,4.048583,3.803728,3.538959,3.240783
75%,43.000000,24.625856,2.000000,1.000000,2.000000,2.000000,4.000000,2.000000,4.000000,2.000000,...,4.000000,2.000000,62.000000,1.000000,1.000000,50.000000,4.460303,4.203006,3.935184,3.643782
max,70.000000,47.184647,4.000000,1.000000,3.000000,3.000000,5.000000,4.000000,5.000000,4.000000,...,4.000000,4.000000,140.000000,1.000000,1.000000,120.000000,5.868545,5.434783,5.482718,5.176030


In [161]:
data.head()

,age,bmi,endurancecat,female,footwear,injury,k5_di,k5_tr,k10_di,k10_tr,...,mf_di,mf_tr,max,sprint,tempo,typical,k5_s,k10_s,mh_s,mf_s
id,,,,,,,,,,,,,,,,,,,,,
4,35.0,23.592323,2,0,2,2,3.0,1.0,5.0,1.0,...,4.0,1.0,60.0,1,1,50.0,4.651163,4.683841,NaN,4.098592
7,33.0,22.518295,2,0,2,2,4.0,3.0,NaN,NaN,...,3.0,2.0,50.0,0,0,40.0,3.869969,NaN,3.662760,3.432720
12,38.0,25.560312,1,0,2,1,NaN,NaN,2.0,3.0,...,4.0,1.0,65.0,0,1,50.0,NaN,3.875969,NaN,3.842896
13,34.0,22.607931,1,0,2,1,NaN,NaN,NaN,NaN,...,3.0,2.0,88.0,1,1,80.0,NaN,NaN,4.223724,3.945670
14,39.0,24.974836,3,0,2,1,4.0,1.0,NaN,NaN,...,2.0,1.0,51.0,0,1,40.0,4.091653,NaN,3.585571,3.136708


In [162]:
# Lazily normalise all data together
# Skip for now
#data = data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))

In [163]:
data.head()

,age,bmi,endurancecat,female,footwear,injury,k5_di,k5_tr,k10_di,k10_tr,...,mf_di,mf_tr,max,sprint,tempo,typical,k5_s,k10_s,mh_s,mf_s
id,,,,,,,,,,,,,,,,,,,,,
4,35.0,23.592323,2,0,2,2,3.0,1.0,5.0,1.0,...,4.0,1.0,60.0,1,1,50.0,4.651163,4.683841,NaN,4.098592
7,33.0,22.518295,2,0,2,2,4.0,3.0,NaN,NaN,...,3.0,2.0,50.0,0,0,40.0,3.869969,NaN,3.662760,3.432720
12,38.0,25.560312,1,0,2,1,NaN,NaN,2.0,3.0,...,4.0,1.0,65.0,0,1,50.0,NaN,3.875969,NaN,3.842896
13,34.0,22.607931,1,0,2,1,NaN,NaN,NaN,NaN,...,3.0,2.0,88.0,1,1,80.0,NaN,NaN,4.223724,3.945670
14,39.0,24.974836,3,0,2,1,4.0,1.0,NaN,NaN,...,2.0,1.0,51.0,0,1,40.0,4.091653,NaN,3.585571,3.136708


In [164]:
# Fillna
data.fillna(value=0, inplace=True)
data.head()

,age,bmi,endurancecat,female,footwear,injury,k5_di,k5_tr,k10_di,k10_tr,...,mf_di,mf_tr,max,sprint,tempo,typical,k5_s,k10_s,mh_s,mf_s
id,,,,,,,,,,,,,,,,,,,,,
4,35.0,23.592323,2,0,2,2,3.0,1.0,5.0,1.0,...,4.0,1.0,60.0,1,1,50.0,4.651163,4.683841,0.000000,4.098592
7,33.0,22.518295,2,0,2,2,4.0,3.0,0.0,0.0,...,3.0,2.0,50.0,0,0,40.0,3.869969,0.000000,3.662760,3.432720
12,38.0,25.560312,1,0,2,1,0.0,0.0,2.0,3.0,...,4.0,1.0,65.0,0,1,50.0,0.000000,3.875969,0.000000,3.842896
13,34.0,22.607931,1,0,2,1,0.0,0.0,0.0,0.0,...,3.0,2.0,88.0,1,1,80.0,0.000000,0.000000,4.223724,3.945670
14,39.0,24.974836,3,0,2,1,4.0,1.0,0.0,0.0,...,2.0,1.0,51.0,0,1,40.0,4.091653,0.000000,3.585571,3.136708


In [165]:
# Split train and test
msk = np.random.rand(len(data)) < 0.7
data_train = data[msk]
data_test = data[~msk]
print(len(data_train))
print(len(data_test))

681
248


In [167]:
# Split into x and y
x_train = data_train.drop('mf_s', axis=1)
y_train = pd.DataFrame(data_train['mf_s'])

x_test = data_test.drop('mf_s', axis=1)
y_test = pd.DataFrame(data_test['mf_s'])
